In [15]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [206]:
#計測地点のosm_idを指定
start_osm = 152781248
mid_osm = []
end_osm =  307601946
start_mesh = 50326612314
end_mesh = 50326612413

In [207]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [208]:
getdbname = "kddi_12_new_param"
dbname = "kddi_trkn12_monthly_anbun"
trknName = "trkn_12_grp"
sttime = "2022-04-01 12:00:00"

In [209]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値50.04444444447004距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値50.355555555588964距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値49.733333333333334距離0.0 km
50326612413
存在メッシュ：50326612413値50.66666666666667距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値50.355555555588964距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値48.63888888891128距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値48.911111111140336距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値48.36666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値49.18333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値48.911111111140336距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値53.12777777785409距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値54.055555555655175距離0.289217697

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値52.316666666733845距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値53.133333333421014距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値51.5距離0.0 km
50326612413
存在メッシュ：50326612413値53.95距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値53.133333333421014距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値52.1444444445139距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値52.98888888897955距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値51.3距離0.0 km
50326612413
存在メッシュ：50326612413値53.833333333333336距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値52.98888888897955距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値51.21666666673658距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値52.066666666757925距離0.2892176975888725 km
50326612314
存在メッシュ：5032661231

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値54.82777777779651距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値55.05555555558001距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値54.6距離0.0 km
50326612413
存在メッシュ：50326612413値55.28333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値55.05555555558001距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値51.06666666667489距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値51.1666666666774距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値50.96666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値51.266666666666666距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値51.1666666666774距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値48.80555555564649距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値49.91111111122981距離0.2892176975888725 km
503266123

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値51.65762711866525距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値51.91525423731579距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値51.4距離0.0 km
50326612413
存在メッシュ：50326612413値52.1728813559322距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値51.91525423731579距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値50.55000000006717距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値51.36666666675435距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値49.733333333333334距離0.0 km
50326612413
存在メッシュ：50326612413値52.18333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値51.36666666675435距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値55.0388888889058距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値55.24444444446652距離0.2892176975888725 km
503266123

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値53.550000000053465距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値54.200000000069785距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値52.9距離0.0 km
50326612413
存在メッシュ：50326612413値54.849999999999994距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値54.200000000069785距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値54.505555555608105距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値55.144444444513034距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値53.86666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値55.78333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値55.144444444513034距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値52.40000000000274距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値52.43333333333691距離0.2892176975888725 km
5

...

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値50.34444444444536距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値50.35555555555675距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値50.333333333333336距離0.0 km
50326612413
存在メッシュ：50326612413値50.36666666666667距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値50.35555555555675距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値51.745009416213264距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値51.956685499081104距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値51.53333333333333距離0.0 km
50326612413
存在メッシュ：50326612413値52.1683615819209距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値51.956685499081104距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値50.41111111114767距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値50.855555555603274距離0.28921769758

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値53.42777777784861距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値54.28888888898135距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値52.56666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値55.150000000000006距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値54.28888888898135距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値49.25555555556561距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値49.3777777777909距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値49.13333333333333距離0.0 km
50326612413
存在メッシュ：50326612413値49.5距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値49.3777777777909距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値50.82222222223228距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値50.94444444445757距離0.2892176975888725 km
503266123

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値50.04444444447004距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値50.355555555588964距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値49.733333333333334距離0.0 km
50326612413
存在メッシュ：50326612413値50.66666666666667距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値50.355555555588964距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値48.63888888891128距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値48.911111111140336距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値48.36666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値49.18333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値48.911111111140336距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値53.12777777785409距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値54.055555555655175距離0.289217697

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値52.316666666733845距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値53.133333333421014距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値51.5距離0.0 km
50326612413
存在メッシュ：50326612413値53.95距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値53.133333333421014距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値52.1444444445139距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値52.98888888897955距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値51.3距離0.0 km
50326612413
存在メッシュ：50326612413値53.833333333333336距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値52.98888888897955距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値51.21666666673658距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値52.066666666757925距離0.2892176975888725 km
50326612314
存在メッシュ：5032661231

33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値54.82777777779651距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値55.05555555558001距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値54.6距離0.0 km
50326612413
存在メッシュ：50326612413値55.28333333333333距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値55.05555555558001距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値51.06666666667489距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値51.1666666666774距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値50.96666666666667距離0.0 km
50326612413
存在メッシュ：50326612413値51.266666666666666距離0.4338265463600201 km
50326612324
存在メッシュ：50326612324値51.1666666666774距離0.2892176975888725 km
33.8473958335:33.8473958335:132.777343750:132.777343750
50326612323
存在メッシュ：50326612323値48.80555555564649距離0.14460884879856797 km
50326612324
存在メッシュ：50326612324値49.91111111122981距離0.2892176975888725 km
503266123

In [210]:
connection.close()

In [205]:
#CreateGeojsonFile